In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Combine.csv')
print(df)

    Unnamed: 0 ?CustomerID        CustomerName         Region  SignupDate  \
0            1       C0001    Lawrence Carroll  South America  10-07-2022   
1            2       C0002      Elizabeth Lutz           Asia  13-02-2022   
2            3       C0003      Michael Rivera  South America  07-03-2024   
3            4       C0004  Kathleen Rodriguez  South America  09-10-2022   
4            5       C0005         Laura Weber           Asia  15-08-2022   
..         ...         ...                 ...            ...         ...   
95          96       C0096    Benjamin Mcclure  South America  18-11-2022   
96          97       C0097           Tina Ford           Asia  18-12-2023   
97          98       C0098       Laura Gilbert  North America  21-10-2024   
98          99       C0099        Rodney Eaton  South America  29-01-2023   
99         100       C0100       Clinton Gomez         Europe  16-03-2023   

   ?ProductID              ProductName     Category   Price  
0        P001

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('Combine.csv')

# Handle missing values (if any)
df = df.dropna()

# Label encoding for categorical data like Region, Category
label_encoder = LabelEncoder()

# Encode 'Region' and 'Category'
df['Region'] = label_encoder.fit_transform(df['Region'])
df['Category'] = label_encoder.fit_transform(df['Category'])

# One-hot encoding for 'ProductName' (if we want to treat product features)
df = pd.get_dummies(df, columns=['ProductName', 'Category'], drop_first=True)

# Feature engineering: Extract signup duration in days
df['SignupDate'] = pd.to_datetime(df['SignupDate'], format='%d-%m-%Y')
df['SignupDuration'] = (pd.to_datetime('today') - df['SignupDate']).dt.days

# Create the customer-product interaction matrix (customer as rows, product as columns)
customer_product_df = df.pivot_table(index='?CustomerID', columns='?ProductID', values='Price', aggfunc='sum', fill_value=0)

# Normalize the interaction matrix if necessary
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
customer_product_scaled = scaler.fit_transform(customer_product_df)

# Calculate cosine similarity to find similar customers
similarity_matrix = cosine_similarity(customer_product_scaled)

# Example: Find similar customers to a specific customer (e.g., Customer 'C0001')
customer_index = customer_product_df.index.get_loc('C0001')
similar_customers = similarity_matrix[customer_index]

# Sort the customers based on similarity and get the top 5 lookalike customers
lookalike_customers = customer_product_df.index[np.argsort(similarity_matrix[customer_index])[::-1][:5]]
print("Lookalike customers for C0001:", lookalike_customers)


Lookalike customers for C0001: Index(['C0001', 'C0020', 'C0006', 'C0021', 'C0017'], dtype='object', name='?CustomerID')
